In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from collections import Counter

In [ ]:
dtr=pd.read_csv('../input/preprocessed/newdf2.csv')
dte=pd.read_csv('../input/preprocessed/newtest.csv')

In [ ]:
Counter(dtr['isFraud'])

In [ ]:
test=pd.read_csv('../input/its-a-fraud/test.csv')

In [ ]:
dtr.drop(['Unnamed: 0'],axis=1,inplace=True)
dte.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
X_train=dtr.drop(['isFraud'],axis=1,inplace=False)
y_train=dtr['isFraud']

In [ ]:
X_test=dte.copy()

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(0.75,random_state=42)
X_res, y_res =  smote.fit_resample(x,y)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression(random_state=0,penalty='none',max_iter=500,class_weight='balanced').fit(X_res, y_res)

In [ ]:
df_res=clf.predict(X_test)

In [ ]:
dte['IsFraud']=df_res

In [ ]:
neww=dte[['TransactionID','IsFraud']]

In [ ]:
test = test.merge(neww, how='outer',copy=False, on ='TransactionID' )

In [ ]:
output = test['IsFraud']

In [ ]:
from collections import Counter

In [ ]:
Counter(df_res)

In [ ]:
output.columns=['Id','isFraud']

In [ ]:
print(output)

In [ ]:
output=pd.DataFrame(output)

In [ ]:
output.head()

In [ ]:
output.columns

In [ ]:
output.to_csv('output.csv',index=True,index_label='Id')

**Logistic Regression using SGD Classifier**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import SGDClassifier

In [ ]:
x = dtr.drop(['isFraud'],axis=1,inplace=False)
y = dtr['isFraud']

In [ ]:
x_train1,x_test1,y_train1,y_test1 = train_test_split(x,y,test_size=0.33, random_state=42)

In [ ]:
type(x_test1)

In [ ]:
sc = StandardScaler()
x_train1_sc = sc.fit_transform(x_train1)
x_test1_sc = sc.transform(x_test1)

In [ ]:
clf1 = SGDClassifier(loss="log")
clf1.fit(x_train1_sc,y_train1)
y_train1_pred=clf1.predict_proba(x_train1_sc)
y_test1_pred = clf1.predict_proba(x_test1_sc)

In [ ]:
print('train auc with default parameter:',roc_auc_score(y_train1,y_train1_pred[:,1]))
print('test auc with default parameter:',roc_auc_score(y_test1,y_test1_pred[:,1]))

In [ ]:
df_res1=clf1.predict(X_test)

In [ ]:
print(df_res1)

In [ ]:
dte['IsFraud']=df_res1

In [ ]:
neww=dte[['TransactionID','IsFraud']]

In [ ]:
test = test.merge(neww, how='outer',copy=False, on ='TransactionID' )

In [ ]:
output = test['IsFraud']

In [ ]:
Counter(df_res1)

In [ ]:
output.columns=['Id','isFraud']

In [ ]:
output=pd.DataFrame(output)
output.to_csv('output.csv',index=True,index_label='Id')

For test, the default parameters are giving score of 0.46349 which is not a good one <br>
We will try tuning the hyperparameters

In [ ]:
alpha = [10**-2,10**-1,10**0,10**1,10**2]
penalty = ['l1','l2','elasticnet']

for i in alpha:
    for j in penalty:
        print('*******alpha=',i,', penalty=',j)
        clf1 = SGDClassifier(loss="log",class_weight='balanced',alpha=i,penalty=j)
        clf1.fit(x_train1_sc,y_train1)
        y_train1_pred=clf1.predict_proba(x_train1_sc)
        y_test1_pred = clf1.predict_proba(x_test1_sc)
        print('train auc:',roc_auc_score(y_train1,y_train1_pred[:,1]))
        print('test auc:',roc_auc_score(y_test1,y_test1_pred[:,1]))
        print('==================================')

We can see that the score is maximum when alpha=0.01 and penalty=l2

In [ ]:
Counter(y)

In [ ]:
clf1 = LogisticRegression(class_weight='balanced',alpha=0.1,penalty='l2')
clf1.fit(X_res,y_res)

In [ ]:
y_test1_pred = clf1.predict(dte)

In [ ]:
Counter(y_test1_pred)

In [ ]:
dte['isFraud']=y_test1_pred

In [ ]:
y_train1_pred=clf1.predict_proba(x_train1_sc)
y_test1_pred = clf1.predict_proba(x_test1_sc)

In [ ]:
print('train auc with default parameter:',roc_auc_score(y_train1,y_train1_pred[:,1]))
print('test auc with default parameter:',roc_auc_score(y_test1,y_test1_pred[:,1]))

In [ ]:
df_res1=clf1.predict(X_test)

In [ ]:
dte['isFraud']=df_res1

In [ ]:
neww=dte[['TransactionID','isFraud']]

In [ ]:
test = test.merge(neww, how='outer',copy=False, on ='TransactionID' )

In [ ]:
output = test['isFraud']

In [ ]:
Counter(df_res1)

In [ ]:
output.columns=['Id','isFraud']

In [ ]:
output=pd.DataFrame(output)
output.to_csv('output.csv',index=True,index_label='Id')